In [ ]:
!pip install flashtext
!pip install s3fs

In [38]:
!pip install presidio-analyzer presidio-anonymizer

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install s3fs
!pip install wget
!pip install certifi
!pip install protobuf==3.20
!pip install pandas==1.1.5
!pip install polyfuzz[all]
!pip install yake==0.4.8
!pip install spacy==3.2.0
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.2.0/en_core_web_lg-3.2.0.tar.gz
!pip install spark-nlp==3.2.1 
!pip install pyspark==3.1.1
!pip install presidio-analyzer==2.2.27
!pip install stanza==1.4.0
!pip install rake-nltk==1.0.6
!pip install nltk==3.6.3
!pip install distributed==2022.2.0
!pip install dask[dataframe]==2022.2.0
!pip install numpy==1.20.0
!pip install beautifulsoup4==4.11.1
!pip install openpyxl==3.0.9
!pip install summa==1.2.0
!pip install tokenizers==0.12.1
!pip install boto3
!pip install torch
!pip install transformers
!pip install scikit-learn
!pip install BeautifulSoup4
!pip install sentence_transformers
!pip install lxml
!pip install pyjarowinkler
!pip install editdistance
!pip install fuzzywuzzy
!pip install rapidfuzz presidio-anonymizer

In [ ]:
!pip install openpyxl

In [4]:
import os
import io
import time
import re
import boto3
import s3fs
import json
import html
import nltk
import datetime
import yake
import random
import spacy
import requests
import logging
import traceback
import pandas as pd
import numpy as np
import warnings
from summa import keywords
warnings.filterwarnings('ignore')
from polyfuzz import PolyFuzz
from rapidfuzz import fuzz
from nltk.tokenize import word_tokenize
from itertools import groupby
from rake_nltk import Rake
from bs4 import BeautifulSoup
from flashtext import KeywordProcessor
from polyfuzz.models import TFIDF,RapidFuzz,Embeddings,EditDistance


In [39]:
text = "Hi Nicole, Hope you are having a great week so far! I wanted to reach out and see if you were interested in attending our upcoming program at Jay's Bistro. Dr. Ammon Larsen will be presenting and answering questions, valuable resources. Please see below for program location and information. Hope to see you there!   Date and time: Thursday, June 29, 2023 6:30 PM Location: Jay's Bistro 135 West Oak Street Fort Collins, CO"

In [51]:
text = ["Hey Sarah, you should stop by David's Bakery on Main Street for some fresh bread.",
"If you're in the mood for a treat, head over to Sarah's Café on Oak Avenue – they have the best pastries, David!",
"I recommend you take your car to John's Auto Repair on Maple Road, Michael, if it needs servicing.",
"How about we meet at Emily's Bookstore near the park to grab some new books, Linda?",
"You should check out Rachel's Floral Shop on River Drive, Jessica – they have gorgeous flowers.",
"I heard Tom's Electronics on Elm Street is having a big sale this weekend, Robert. You might want to visit!",
"If you need any alterations, Mike's Tailoring on the corner of 5th and Pine is the place to go, Mark.",
"I bet you'd love Laura's Boutique on Queen's Lane, Emma. They have some really stylish clothes.",
"You should grab a coffee at Anna's Espresso Bar on Broadway next time you're in the area, Jake.",
"If you're looking for a new gym, Jack's Gym on 2nd Avenue just opened a fantastic fitness center, Olivia.",
"Hi Nicole, Hope you are having a great week so far! I wanted to reach out and see if you were interested in attending our upcoming program at Jay's Bistro. Dr. Ammon Larsen will be presenting and answering questions, valuable resources. Please see below for program location and information. Hope to see you there!   Date and time: Thursday, June 29, 2023 6:30 PM Location: Jay's Bistro 135 West Oak Street Fort Collins, CO"
]

In [67]:
data_file = 'C:\\Users\\C289588\\Downloads\\hcp\\False_Positive_Analysis\\check_keywords_both_all_data.xlsx'

In [68]:
import pandas as pd
data = pd.read_excel(data_file)

In [40]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\C289588\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
!pip install nltk


Defaulting to user installation because normal site-packages is not writeable


Presidio

In [90]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine


anonymizer = AnonymizerEngine()

# Initialize Presidio analyzer
analyzer = AnalyzerEngine()

# Sample text containing name and address
#text = "John Doe lives at 1234 Elm Street, Springfield, IL 62704."

# Analyze the text for entities
results = analyzer.analyze(text=text, entities=["PERSON", "LOCATION"], language='en')

# Print results
for result in results:
    #print(f"Entity: {result.entity_type}, Value: {result.text}")
    print(f"Entity: {result.entity_type}, Value: {result.contains}")

anonymized_text = anonymizer.anonymize(
       text=text,
       analyzer_results=results
   )
anonymized_text.text

Entity: PERSON, Value: <bound method RecognizerResult.contains of type: PERSON, start: 3, end: 9, score: 0.85>
Entity: PERSON, Value: <bound method RecognizerResult.contains of type: PERSON, start: 160, end: 172, score: 0.85>


"Hi <PERSON>, Hope you are having a great week so far! I wanted to reach out and see if you were interested in attending our upcoming program at Jay's Bistro. Dr. <PERSON> will be presenting and answering questions, valuable resources. Please see below for program location and information. Hope to see you there!   Date and time: Thursday, June 29, 2023 6:30 PM Location: Jay's Bistro 135 West Oak Street Fort Collins, CO"

NLTK

In [31]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\C289588\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\C289588\AppData\Roaming\nltk_data...


True

In [32]:
from nltk.corpus import wordnet as wn

def disambiguate_word(word):
    # Look up the word in WordNet
    synsets = wn.synsets(word)

    if not synsets:
        return None

    # For simplicity, we use the first sense
    return synsets[0].lemma_names()

def identify_place_or_person(text):
    words = text.split()
    places_or_people = []

    for word in words:
        word_senses = disambiguate_word(word)

        if word_senses:
            # Look for clues in WordNet senses
            if "person" in word_senses[0].lower():
                places_or_people.append((word, "Person"))
            elif "location" in word_senses[0].lower():
                places_or_people.append((word, "Place"))
            else:
                places_or_people.append((word, "Uncertain"))
        else:
            places_or_people.append((word, "Uncertain"))

    return places_or_people

# Example sentences
sentence1 = "Meet me at David's Restaurant."
sentence2 = "Meet David at the restaurant."

print(identify_place_or_person(sentence1))
print(identify_place_or_person(sentence2))


[('Meet', 'Uncertain'), ('me', 'Uncertain'), ('at', 'Uncertain'), ("David's", 'Uncertain'), ('Restaurant.', 'Uncertain')]
[('Meet', 'Uncertain'), ('David', 'Uncertain'), ('at', 'Uncertain'), ('the', 'Uncertain'), ('restaurant.', 'Uncertain')]


In [14]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tree import Tree

# Ensure necessary resources are downloaded
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Sample text
text = "John Doe lives at 1234 Elm Street, Springfield, IL 62704."

# Tokenize the text
words = word_tokenize(text)

# Tag the words with parts of speech
tagged = pos_tag(words)

# Perform named entity recognition
tree = ne_chunk(tagged)

# Extract named entities
for subtree in tree:
    if isinstance(subtree, Tree):
        entity = " ".join([word for word, tag in subtree])
        label = subtree.label()
        if label == "GPE":
            print(f"Address-like Entity: {entity}")
        elif label == "PERSON":
            print(f"Name: {entity}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\C289588\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\C289588\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\C289588\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\C289588/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\C289588\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
*****************

In [7]:
for result in results:
    print(dir(result))

['IS_SCORE_ENHANCED_BY_CONTEXT_KEY', 'RECOGNIZER_NAME_KEY', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', 'analysis_explanation', 'append_analysis_explanation_text', 'contained_in', 'contains', 'end', 'entity_type', 'equal_indices', 'from_json', 'has_conflict', 'intersects', 'logger', 'recognition_metadata', 'score', 'start', 'to_dict']
['IS_SCORE_ENHANCED_BY_CONTEXT_KEY', 'RECOGNIZER_NAME_KEY', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__

SPACY

In [42]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


In [43]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 4.5 MB/s eta 0:00:03
     --------- ------------------------------ 3.1/12.8 MB 6.1 MB/s eta 0:00:02
     ----------------- ---------------------- 5.5/12.8 MB 7.8 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 9.4 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.9

In [70]:
import spacy

nlp = spacy.load('en_core_web_sm')

def spacy_address(text):
    text = str(text)
    doc = nlp(text)
    address = []
    for entity in doc.ents:
      if entity.label_ in ['ORG', 'FAC', 'GPE'] :
         address.append(entity.text)
    return address
data["address"] = data['Lilly_User_Input_Text_1__c'].apply(spacy_address)

In [71]:
data.to_csv("address_check.csv")

In [56]:
import spacy

nlp = spacy.load('en_core_web_sm')

#text = "I love traveling to Paris and London. I also enjoy visiting Sydney."
for i in text:
   print(i)
   doc = nlp(i)

   for entity in doc.ents:
      #print(entity.label_)
      if entity.label_ in ['ORG', 'FAC', 'GPE'] :
         print(entity.text)

Hey Sarah, you should stop by David's Bakery on Main Street for some fresh bread.
If you're in the mood for a treat, head over to Sarah's Café on Oak Avenue – they have the best pastries, David!
I recommend you take your car to John's Auto Repair on Maple Road, Michael, if it needs servicing.
John's Auto Repair on Maple Road
How about we meet at Emily's Bookstore near the park to grab some new books, Linda?
Emily's Bookstore
You should check out Rachel's Floral Shop on River Drive, Jessica – they have gorgeous flowers.
Rachel's Floral Shop on River Drive
I heard Tom's Electronics on Elm Street is having a big sale this weekend, Robert. You might want to visit!
Tom's Electronics
Elm Street
If you need any alterations, Mike's Tailoring on the corner of 5th and Pine is the place to go, Mark.
Pine
I bet you'd love Laura's Boutique on Queen's Lane, Emma. They have some really stylish clothes.
Laura's Boutique on Queen's Lane, Emma
You should grab a coffee at Anna's Espresso Bar on Broadway 

TRANSFORMERS

In [47]:
!pip install transformers
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [50]:
from transformers import pipeline

# Load a pre-trained NER model
nlp_ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def identify_place_or_person(text):
    # Use the NER model to get entities
    entities = nlp_ner(text)

    places_or_people = []

    for entity in entities:
        #print(entity)
        if entity['entity'] == "B-ORG" or entity['entity'] == "I-ORG":
            places_or_people.append((entity['word'], "PLace"))
        elif entity['entity'] == "B-LOC" or entity['entity'] == "I-LOC":
            places_or_people.append((entity['word'], "Place"))

    return places_or_people

# Example sentences
#sentence1 = "Meet me at David's Restaurant."
#sentence2 = "Meet David at the restaurant."

print(identify_place_or_person(text))
#print(identify_place_or_person(sentence2))


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[('Jay', 'PLace'), ("'", 'PLace'), ('s', 'PLace'), ('B', 'PLace'), ('##ist', 'PLace'), ('##ro', 'Place'), ('Jay', 'PLace'), ("'", 'PLace'), ('s', 'PLace'), ('B', 'PLace'), ('##ist', 'PLace'), ('##ro', 'PLace'), ('West', 'Place'), ('Oak', 'Place'), ('Fort', 'Place'), ('Collins', 'Place'), ('CO', 'Place')]


In [ ]:
with open("processing_job_config.json","r") as cf:
        config_file=json.load(cf)
    
bucket_name = config_file["bucket_name"]
params = config_file["dictionary_model"]
dictionary_path = params["dictionary_path"]
nr_candidates = params["nr_candidates"]
manual_restricted_words = config_file["manual_restricted_words"]
#content = s3.get_object(Bucket=bucket_name, Key=manual_restricted_words)['Body']
#manual_dictionary=json.loads(content.read())
restricted_wrds_file = config_file["restricted_words"]
competitor_names = dictionary_path+'hcp_cancrgov_mods_competitor_nm.json'
drugbank_product = dictionary_path+'hcp_drugbnk_mods_ctgry_prdct.json'
medra_disease_names = dictionary_path+'hcp_medra_disease_nm.json'
product_attributes = dictionary_path+'hcp_product_attributes.json'

#obj = s3.get_object(Bucket=bucket_name, Key=restricted_wrds_file)
df_restricted_wrd = pd.read_excel('rwl_category_veeva.xlsx',engine='openpyxl')

In [8]:
!pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.5 MB 5.6 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/11.5 MB 8.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.5 MB 7.5 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.5 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2


  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\C289588\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
with open('manual_restricted_words.json', 'r') as file:
        manual_dictionary = json.load(file)
manual_trigger_words = manual_dictionary["restricted_words"]

In [25]:
def convert_into_str(cols):
    #print(cols[0])
    if(cols[0])==1:
        return cols["restricted_words"]

In [ ]:
def create_restricted_json(raw_json, catg):
    restricted_keywords = {}
    for key in catg:
        temp=[]
        for inner_key , val in raw_json[key].items():
            if val!=None:
                temp.append(val)
        if temp:
            restricted_keywords.update({key:list(set(temp))})
    return restricted_keywords

In [67]:
import spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     --------------------------------------- 3.7/587.7 MB 31.3 MB/s eta 0:00:19
      ------------------------------------- 10.5/587.7 MB 32.8 MB/s eta 0:00:18
      ------------------------------------- 10.5/587.7 MB 32.8 MB/s eta 0:00:18
      ------------------------------------- 14.7/587.7 MB 19.6 MB/s eta 0:00:30
     - ------------------------------------ 21.0/587.7 MB 22.9 MB/s eta 0:00:25
     - ------------------------------------ 24.4/587.7 MB 21.2 MB/s eta 0:00:27
     - ------------------------------------ 30.4/587.7 MB 22.4 MB/s eta 0:00:25
     -- ----------------------------------- 33.8/587.7 MB 21.7 MB/s eta 0:00:26
     -- ----------------------------------- 40.9/587.7 MB 23.2 MB/s eta 0:00:24
     -- ----------------------------------- 44.8/587.7 MB 22.6 MB/s eta 0:00:24
     --- ---------------------------------- 50.6/


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\C289588\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [69]:
processor = KeywordProcessor(case_sensitive = False)
nlp = spacy.load("en_core_web_lg")
all_stopwords = nlp.Defaults.stop_words

In [ ]:
def preprocess_restricted_wrd(text, df_restricted_wrd, manual_trigger_words):
    df_col = ["RowSum","Other"]
    for colnames in df_col:
        if colnames in  list(df_restricted_wrd.columns):
            df_restricted_wrd.drop(columns=[colnames],inplace=True)
    inputcols = df_restricted_wrd.columns[1:]
    #print(df_restricted_wrd)
    #print(inputcols)
    for i in inputcols:
        df_restricted_wrd[i] = df_restricted_wrd[[i,"restricted_words"]].apply(convert_into_str,axis=1)
    #print(df_restricted_wrd)
    restricted_json = df_restricted_wrd.to_dict()
    #print(restricted_json)
    catg = list(restricted_json.keys())[1:]
    #print(catg)
    restricted_catg_kywrds = create_restricted_json(restricted_json, catg)
    #print(restricted_catg_kywrds)
    for key, val in manual_trigger_words.items():
        if key in restricted_catg_kywrds.keys():
            temp=list(set(restricted_catg_kywrds[key]+val))
            restricted_catg_kywrds[key] = temp
        else:
            restricted_catg_kywrds.update({key:val})
    #print(restricted_catg_kywrds)
    restricted_kywrds = [wrd for key, val in restricted_catg_kywrds.items() for wrd in val]
    #print(restricted_kywrds)
    restricted_kywrds = [wd for wd  in restricted_kywrds if len(wd)>=4]
    #print(restricted_kywrds)
    processor.add_keywords_from_list(restricted_kywrds)
    found = processor.extract_keywords(str(text))
    print(found)
    direct_match = [wrd for wrd in restricted_kywrds if (len(wrd.strip().split(" "))>1) and (wrd.strip().lower() in str(text).lower())]
    found.extend(direct_match)
    print(direct_match)
    found = list(set(found))
    flashtext_res = [(wrd,100.0) for wrd in found ]
    print(flashtext_res)
    return restricted_catg_kywrds, {"restricted_wrds":restricted_kywrds}, flashtext_res

In [59]:
preprocess_restricted_wrd(text, df_restricted_wrd, manual_trigger_words)

['valuable resources']
['valuable resources']
[('valuable resources', 100.0)]


({'Adjectives in relation to any product, indication, or clinical data': ['confident will provide value',
   'difficult patient solution',
   'exciting development',
   'key update',
   'key product information',
   'most up-to-date product',
   'information below is an exciting time',
   'recent fda approval',
   'recent product indication',
   'great patient education resource',
   'nice update',
   'key product updates',
   'exciting updated information',
   'important product updates and information',
   'exciting advancement',
   'important update',
   'important information',
   'exciting information',
   'important data',
   'important product update',
   'valuable resources',
   'critical information',
   'landmark update',
   'newest',
   'landmark data',
   'exciting info',
   'valuable information',
   'important indication',
   'landmark info',
   'exciting indication',
   'updated indication',
   'recent indication',
   'latest indication',
   'important key update',
   'i

In [73]:
def decontracted(phrase):
    """
    Standardize certain tokens into a general format
    """
    phrase=re.sub("’","'",phrase) 
    phrase=re.sub('“|”','"',phrase)
    #phrase=BeautifulSoup(html.unescape(phrase.encode("cp1252",errors='ignore').decode("utf-8")), "lxml").text#using a cp1252 encding as there are appearances of certain characters not supported by utf
    phrase=BeautifulSoup(html.unescape(phrase.encode("utf-8",errors='ignore').decode("utf-8")), "lxml").text
    phrase=re.sub("’","'",phrase) 
    phrase=re.sub('“','"',phrase)
    phrase=re.sub("\s+"," ",phrase)
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"y\'all", "you all", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    #phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
def kwe_rake(text,model_rake):# Rake model which extracts keyphrases on basis of occurance of stop words
    kw_list=list()
    text=re.sub(r"'","",text)
    model_rake.extract_keywords_from_text(text)
    phrases=model_rake.get_ranked_phrases()
    return extract_unique_phrases(phrases)

def kwe_yake(text,yake_kw_extractor): #Yake model to extract keyphrases with max length as 8 using statistical properties
    kw_list=list()
    keywords = yake_kw_extractor.extract_keywords(text)
    return extract_unique_phrases(keywords)

def kwe_textrank(text):#Graph based solution to extract keyphrases
    text_tokens = word_tokenize(text)
    sent_without_sw =" ".join([word for word in text_tokens if word.lower() not in all_stopwords])
    textrank_phrases = keywords.keywords(sent_without_sw,ratio=1,split=True)
    return extract_unique_phrases(textrank_phrases)
def custom_extraction(text):
    """
    Custom model that will extract phrases based on patterns found in text using spacy and custom logic
    """
    res=list()
    response=list()
    counter=0
    doc=nlp(text)
    customlist=["year","email","phone","name","time","month","let","know","like","weeks"] #addtional stopwords which commonly occur
    temp=[(tk.text,tk.pos_) for tk in doc]#Token and its POS Tag
    wrds=[i[0] for i in temp] #tokens in a text
    pos=["#" if (i[1]=="NOUN" or i[1]=="PROPN") else i[1] for i in temp] #tags changed to "#" for normalizing in Tags
    for e,t in enumerate(groupby(pos)): #groupby to get all consecutive NOUN/PROPN i.e "#" of any combination together
        lst=list(t[1])
        if t[0]=="#":
            val=[(e1+counter) for e1,_ in enumerate(lst)]
            res.append(val)
        if len(lst)>1:
            counter+=(len(lst))
        else:
            counter+=1
    for phrase in [(" ".join(wrds[i[0]:i[-1]+1])).strip() for i in res]:
        if phrase not in response:
            for wd in (list(all_stopwords)+customlist):
                exp=r"\b"+(wd)+"\\b|\\b"+(wd)+"s"+"\\b"
                phrase=re.sub(exp," ",phrase.lower()) 
                phrase=re.sub("\s+"," ",re.sub("[\(\)\[\]\?!=:;/\\\_-]"," ",phrase)).strip()
            if (phrase) and (phrase not in response):
                response.append(phrase)
    adjwrds=[adj[0] for adj in temp if adj[1]=="ADJ"]
    response.extend(adjwrds)
    return response
def extract_unique_phrases(keywords):#Common function to get only unique phrases
    """
    Keywords->All keyphrases extracted from text using textrank,rake,yake
    """
    kw_list=list()
    for k in keywords:
        if type(k)!=str:
            k=k[0]
        keywrd=k.lower().strip()
        if keywrd not in kw_list:
            kw_list.append(keywrd)
    return kw_list

In [ ]:
tfidf = TFIDF(n_gram_range=params["tfidf_ngram_range"],min_similarity=params["kw_cut_off"],model_id="TFIDF",cosine_method="sparse")
rapid = RapidFuzz(n_jobs=1,score_cutoff=params["kw_cut_off"],scorer=fuzz.token_sort_ratio,model_id="RAPID")
    
#Initialize models for keyphrase extraction
rk = Rake(stopwords=all_stopwords,punctuations=None)
#print(rk)
yake_kw_extractor = yake.KeywordExtractor(stopwords=all_stopwords,lan='en', n=params["max_ngram"][1], dedupLim=params["deduplication_threshold"], windowsSize=params["window"])
#print(yake_kw_extractor)
cleanedtext=decontracted(text)
print(cleanedtext)
rake=kwe_rake(cleanedtext,rk)
print(rake)
yake_kw=kwe_yake(cleanedtext,yake_kw_extractor)
print(yake_kw)
textrank=kwe_textrank(cleanedtext)
print(textrank)
custom=custom_extraction(cleanedtext)
print(custom)
merge_cols=list(set(rake+textrank+custom+yake_kw))
print(merge_cols)

Hi Nicole, Hope you are having a great week so far! I wanted to reach out and see if you were interested in attending our upcoming program at Jay's Bistro. Dr. Ammon Larsen will be presenting and answering questions, valuable resources. Please see below for program location and information. Hope to see you there! Date and time: Thursday, June 29, 2023 6:30 PM Location: Jay's Bistro 135 West Oak Street Fort Collins, CO
['jays bistro 135 west oak street fort collins', 'jays bistro', '30 pm location', 'program location', 'valuable resources', 'upcoming program', 'june 29', 'hi nicole', 'great week', 'answering questions', 'ammon larsen', '2023 6', 'wanted', 'time', 'thursday', 'reach', 'presenting', 'interested', 'information', 'hope', 'having', 'far', 'dr', 'date', 'co', 'attending']
['reach out and see if you were interested', 'having a great week so far', 'west oak street fort collins', 'hope you are having a great', 'ammon larsen will be presenting and answering questions', 'nicole', 

In [93]:
import re 
import json
import boto3
import time
from rapidfuzz import fuzz,process
from presidio_analyzer import AnalyzerEngine
#from .namematch import NameMatch

# In[ ]:
s3 = boto3.client('s3')

"""
Get the values from the config file
"""


with open("processing_job_config.json","r") as cf:
    config_file=json.load(cf)

param=config_file["offshelf_model"]
pii_name_cutoff=param["pii_cut_off"]
pii_analyzer = AnalyzerEngine(default_score_threshold=param["pii_analyzer_threshold"])
#entity_list = list(pii_analyzer.get_supported_entities())
entity_list= ["MEDICAL_LICENSE", "PERSON", "US_SSN", "AU_MEDICARE", "NRP", "EMAIL_ADDRESS"]

In [94]:
pii_analyzer

In [95]:
def fuzz_match(full_nm_lst,speaker_lst):
    temp = []
    for nm in full_nm_lst:
        for sp_nm in speaker_lst:
            if (fuzz.partial_ratio(nm,sp_nm)/100)>=pii_name_cutoff:
                temp.append(nm)
                break
    full_nm_lst = [(i,100,"PERSON") for i in full_nm_lst if i not in temp]
    return full_nm_lst

In [83]:
def detect_pii(text,lst_of_sentences,recipient_name,sender_name,speaker_lst,fieldname): #presidio analyzer to detect to Privacy related information
    
    """
    Input Args: Text ->  complete text
           lst_of_sentences -> list of sentences of text obtained using sparknlp
           recipient name -> Name of the recipient got from the VAE
           sender name -> Name of the sender got from the VAE
           
    Returns: 
            (pii entities,entity type,sentences with those entities,score,time taken)
    
    """
    start_time=time.time()
    full_nm = []
    rm_wrds = ["hope","hello"]
    pii_phrases,temp,reciever_names,sender_names,first_name = ([] for i in range(5)) #generate multiple empty lists
    pii_entities = pii_analyzer.analyze(text, entities=entity_list, language='en') #instantiate the pii analyzer
    print(pii_entities)
    for phrase in pii_entities:
        #condition to get all matching name of the reciepient from identified entities
        if (phrase.entity_type == "PERSON" and ((fuzz.partial_token_set_ratio(text[phrase.start:phrase.end].lower(),recipient_name.lower())/100>pii_name_cutoff))):
            reciever_names.append(text[phrase.start:phrase.end].lower())
        #condition to get all matching name of the reciepient from identified entities
        if (phrase.entity_type == "PERSON" and ((fuzz.partial_token_set_ratio(text[phrase.start:phrase.end].lower(),sender_name.lower())/100>pii_name_cutoff))):
            sender_names.append(text[phrase.start:phrase.end].lower())
            
        if (phrase.entity_type == "PERSON") and (fieldname == "Lilly_User_Input_Text_1__c") and  (int(phrase.end)<=20) and (text[phrase.start:phrase.end].lower() not in sender_names):
            
            first_extracted_name = text[phrase.start:phrase.end].lower()
            for wrd in rm_wrds:
                first_extracted_name = re.sub(f'^dr',"",first_extracted_name)
                first_extracted_name = re.sub(f'{wrd}',"",first_extracted_name)
            first_extracted_name = first_extracted_name.lstrip().rstrip()    
            if first_extracted_name:
                first_name.append(first_extracted_name)
            
        #extract all unique  entities from text  using their span
        if text[phrase.start:phrase.end] not in temp:
            ent_pair=(text[phrase.start:phrase.end].lower(),phrase.score,phrase.entity_type)
            pii_phrases.append(ent_pair)
            temp.append(text[phrase.start:phrase.end]) 
    if reciever_names: #remove first occurence of recipient name
        pii_phrases=[i for i in pii_phrases if reciever_names[0]!=i[0]]
    if sender_names: #remove first occurence of sender name
        pii_phrases=[i for i in pii_phrases if sender_names[0]!=i[0]]
    pii_phrases_temp = [i for i in pii_phrases if i[2]=="PERSON"]
    pii_phrases_person = []
    for nm in pii_phrases_temp:
        new_nm = nm[0]
        for wd in rm_wrds:
            new_nm = re.sub("^(dr.|dr )","",new_nm)
            new_nm = re.sub(f'{wd}',"",new_nm)
        new_nm = str(new_nm).lstrip().rstrip()
        if len(new_nm)>2:
            t=(new_nm,nm[1],nm[2])
            pii_phrases_person.append(t)
    pii_phrases = [i for i in pii_phrases if i[2]!="PERSON"]
    
    for nm in pii_phrases_person:
        if(len(re.split(" |,|-",nm[0]))>=2):
            if len(re.split(" |,|-",nm[0])[-1])>1:
                full_nm.append(nm[0])
                
    full_names= fuzz_match(full_nm,speaker_lst)
    pii_phrases.extend(full_names)
    if first_name:
        key = (first_name[0],recipient_name)
        nm = NameMatch(key)
        val = nm.matchname()
        if val == "No Match":
            t=(first_name[0],100,"PERSON")
            pii_phrases.append(t)
    #print(pii_phrases)
    #extract sentences where the entities where present
    res=[(i[0],i[1],i[2],[sent for sent in lst_of_sentences if bool(re.search(r"(^|\W)"+re.escape(i[0])+r"(\W|$)",sent,re.IGNORECASE))])                                                                                                for i in pii_phrases]
    res = [(i[0],2.2,i[2],i[3]) if i[2] == "PERSON" else (i[0],i[1],i[2],i[3]) for i in res ]
    exe_time=time.time()-start_time
    return res, round(exe_time,3) #return response along with the execution time

In [84]:
text = "Hi Nicole, Hope you are having a great week so far! I wanted to reach out and see if you were interested in attending our upcoming program at Jay's Bistro. Dr. Ammon Larsen will be presenting and answering questions, valuable resources. Please see below for program location and information. Hope to see you there!   Date and time: Thursday, June 29, 2023 6:30 PM Location: Jay's Bistro 135 West Oak Street Fort Collins, CO"
lst_of_sentences = ["Hi Nicole, Hope you are having a great week so far","I wanted to reach out and see if you were interested in attending our upcoming program at Jay's Bistro.","Dr. Ammon Larsen will be presenting and answering questions, valuable resources.","Please see below for program location and information.", "Hope to see you there!","Date and time: Thursday, June 29, 2023 6:30 PM Location: Jay's Bistro 135 West Oak Street Fort Collins, CO"]
recipient_name = 'Nicole Kidman'
sender_name = 'David hughman'
speaker_lst = 'Brandy Kole'
fieldname = 'Lilly_User_Input_Text_1__c'
ans = detect_pii(text, lst_of_sentences, recipient_name, sender_name, speaker_lst, fieldname) #presidio analyzer to detect to Privacy related information


[]


In [85]:
ans

([], 0.013)

In [96]:
pii_entities = pii_analyzer.analyze(text, language='en')

In [97]:
pii_entities

[]